In [1]:
import numpy as np
import torch
from mcspace.data_utils import get_data, get_mouse_diet_perturbations_dataset, get_human_timeseries_dataset
from mcspace.utils import get_device, pickle_load, pickle_save, get_summary_results, MODEL_FILE, DATA_FILE
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from mcspace.dataset import DataSet
import pandas as pd

In [2]:
rootpath = Path("../../")
datapath = rootpath / "paper" / "datasets"

In [3]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 1086-9223

 Directory of C:\Users\Gary\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\paper\figures

10/07/2024  06:51 PM    <DIR>          .
09/24/2024  12:35 PM    <DIR>          ..
10/07/2024  06:51 PM    <DIR>          .ipynb_checkpoints
09/24/2024  03:52 PM    <DIR>          _tmp_sub
09/24/2024  03:32 PM           208,580 data_filtering_supplemental.ipynb
10/07/2024  06:50 PM                72 explore_raw_data.ipynb
09/30/2024  12:34 PM           369,059 figure2_experimental_design_dataset_stats.ipynb
09/26/2024  02:13 PM            40,785 figure4_cross_validation.ipynb
09/26/2024  03:21 PM           256,921 figure5_human_results.ipynb
09/26/2024  02:09 PM           281,161 figures_3_S1_semisynthetic_benchmarking.ipynb
10/02/2024  08:12 AM    <DIR>          main_figures
09/27/2024  11:43 AM           427,992 supplemental_data_figure.ipynb
09/24/2024  05:48 PM               609 supplemen

In [5]:
ls "../datasets/human_experiments/"

 Volume in drive C is Windows-SSD
 Volume Serial Number is 1086-9223

 Directory of C:\Users\Gary\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\paper\datasets\human_experiments

09/24/2024  02:43 PM    <DIR>          .
09/24/2024  02:43 PM    <DIR>          ..
08/13/2024  01:12 PM        67,629,503 count_data.csv
09/24/2024  02:43 PM    <DIR>          gappa_taxonomy
09/24/2024  02:43 PM    <DIR>          human_otus_tree
08/13/2024  01:12 PM            57,852 taxonomy.csv
08/13/2024  01:12 PM           316,656 zotus.fa
               3 File(s)     68,004,011 bytes
               4 Dir(s)  76,926,836,736 bytes free


In [7]:
ls "../datasets/mouse_experiments//"

 Volume in drive C is Windows-SSD
 Volume Serial Number is 1086-9223

 Directory of C:\Users\Gary\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\paper\datasets\mouse_experiments

09/24/2024  02:43 PM    <DIR>          .
09/24/2024  02:43 PM    <DIR>          ..
09/24/2024  02:43 PM    <DIR>          .ipynb_checkpoints
06/11/2024  10:10 AM           109,967 combined_zotus .fa
08/13/2024  01:22 PM        12,618,963 mouse_counts.csv.gz
09/24/2024  02:43 PM    <DIR>          output
06/11/2024  12:13 PM           157,797 tax.csv
09/24/2024  02:43 PM    <DIR>          taxonomy
09/24/2024  02:43 PM    <DIR>          tmp
08/26/2024  09:28 AM            21,154 update_taxonomy.ipynb
06/11/2024  10:10 AM           109,967 zotus.fa
               5 File(s)     13,017,848 bytes
               6 Dir(s)  76,927,401,984 bytes free


In [6]:
hdata = pd.read_csv(datapath / "human_experiments" / "count_data.csv")

C:\Users\Gary\AppData\Local\Temp\ipykernel_131004\1056325987.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  hdata = pd.read_csv(datapath / "human_experiments" / "count_data.csv")


In [9]:
mdata = pd.read_csv(datapath / "mouse_experiments" / "mouse_counts.csv.gz", compression='gzip')

C:\Users\Gary\AppData\Local\Temp\ipykernel_131004\1241848784.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mdata = pd.read_csv(datapath / "mouse_experiments" / "mouse_counts.csv.gz", compression='gzip')


In [12]:
hparticles = np.array(hdata['Count'])

In [14]:
hdata['Count'].describe()

count    4.473430e+05
mean     1.298152e+02
std      4.264490e+03
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+01
max      2.613365e+06
Name: Count, dtype: float64

In [21]:
np.percentile(hparticles, q=99.5)

4150.579999999958

In [19]:
mdata['Count'].describe()

count    1.056245e+06
mean     3.766216e+02
std      5.802009e+03
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      6.500000e+01
max      1.707640e+06
Name: Count, dtype: float64

In [22]:
mparticles = np.array(mdata['Count'])

In [23]:
np.percentile(mparticles, q=99)

5547.0

In [25]:
print(sum(hparticles < 250) / len(hparticles))
print(sum(hparticles < 10000) / len(hparticles))

0.9503155296942167
0.9982474298245418


In [26]:
print(sum(mparticles < 250) / len(mparticles))
print(sum(mparticles < 10000) / len(mparticles))

0.8753414217345408
0.9944293227423562
